In [28]:
import sys
from pathlib import Path
import warnings
warnings.filterwarnings("ignore", module="IPython")

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('aurora',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

print(f"Root dir: {root_dir}")

# Add the root directory to the `PYTHONPATH` 
if root_dir not in sys.path:
    sys.path.append(root_dir)
    print(f"Added the following directory to the PYTHONPATH: {root_dir}")

# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Root dir: /Users/appbites/Desktop/id2223-project
HopsworksSettings initialized!


## Imports & Constants

In [29]:
import datetime as datetime
import pandas as pd
from xgboost import XGBClassifier
import hopsworks
import json
from mlfs.aurora import util
import os
import joblib

In [30]:
KP_FG = dict(name="geomagnetic_daily_final", version=1)
WEATHER_FG = dict(name="sweden_weather_daily_final", version=1)

AURORA_FV = dict(name="aurora_fv_final", version=1)

PRED_FG = ("aurora_predictions", 1)

MODEL_NAME = "aurora_xgboost"   # el prefijo que usas en training: f"{MODEL_NAME}_h{h}"
MAX_HORIZON = 5               # o el que uses
AP_THRESHOLD = 15             # tu umbral del evento

DATA_PATH = "../../data"
RUN_DATE = datetime.datetime.utcnow().date()- datetime.timedelta(days=1)

## Hopsworks Login

In [31]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store()    # Feature Store
mr = project.get_model_registry()  # Model Registry

2026-01-05 23:18:47,633 INFO: Closing external client and cleaning up certificates.
Connection closed.
2026-01-05 23:18:47,670 INFO: Initializing external client
2026-01-05 23:18:47,671 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-05 23:18:48,357 WARNING: UserWarning: The installed hopsworks client version 4.4.2 may not be compatible with the connected Hopsworks backend version 4.2.2. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2026-01-05 23:18:49,386 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1289364


## 1. Fetch the Inference Data and Add it to the Feature Store

In [32]:
def fetch_kp():
    # Obtain data
    df = util.get_latest_complete_kp_from_nowcast()

    # Insert into feature store
    kp_fg = fs.get_feature_group(**KP_FG)
    kp_fg.insert(df, wait=True)
    
    return df

In [33]:
def fetch_weather(run_date):

    LATITUDE = 62.0
    LONGITUDE = 15.0

    run_date = pd.to_datetime(run_date).date()
    date_str = run_date.strftime("%Y-%m-%d")

    df = util.get_historical_weather_sweden(
        start_date=run_date - pd.Timedelta(days=7),
        end_date=date_str,
        latitude=LATITUDE,
        longitude=LONGITUDE,
    )

    weather_fg = fs.get_feature_group(**WEATHER_FG)
    weather_fg.insert(df, wait=True)
    
    return df

In [34]:
def build_features(raw: dict) -> pd.DataFrame:
    df = build_feature_dataframe(
        kp=raw["kp"],
        weather=raw["weather"],
    )

    # último timestamp disponible
    df = df.sort_index().iloc[-1:]
    return df

In [35]:
def load_model(model_name: str):
    mr = project.get_model_registry()
    model = mr.get_model(
        name=model_name,
        alias="champion"
    )
    return model.download()

### Steps of the main Loop

In [36]:
# Step 1
def fetch_latest_raw_data():
    # Fetch latest geomagnetic data (includes several recent days)
    kp_df = fetch_kp()

    # Normalize dates
    kp_df["date"] = pd.to_datetime(kp_df["date"]).dt.normalize()

    # Use the latest available date as "today"
    date = kp_df["date"].iloc[-1]

    print("Last data from kp:", date)

    # Fetch weather for the same window (already returns several days)
    weather_df = fetch_weather(date)

    weather_df["date"] = pd.to_datetime(weather_df["date"]).dt.normalize()

    # --- Build 5-day context window ---
    context = []

    # Take last 5 days where both kp + weather exist
    kp_recent = kp_df.tail(5).reset_index(drop=True)

    for _, row in kp_recent.iterrows():
        day = row["date"]

        # Match weather row for the same date
        w_row = weather_df[weather_df["date"] == day]
        if w_row.empty:
            continue

        w_row = w_row.iloc[0]

        context.append({
            "date": str(day.date()),
            "geomagnetic": {
                "ap": float(row["ap"]),
                "kp_max": float(
                    max(row[f"kp{i}"] for i in range(1, 9))
                )
            },
            "weather": {
                "cloud_cover": float(w_row["cloud_cover_mean"]),
                "precipitation": float(w_row["precipitation_sum"])
            }
        })

    # --- Persist context ---
    out_dir = Path("../../docs/aurora/assets")
    out_dir.mkdir(parents=True, exist_ok=True)

    with open(out_dir / "context.json", "w") as f:
        json.dump(context, f, indent=2)

    print(f"Saved {len(context)} days of context to context.json")

    return date


In [37]:
# Step 2

def obtain_data_fv(date):

    date = pd.to_datetime(date, utc=True).normalize()
    
    fv = fs.get_feature_view(**AURORA_FV)

    X = fv.get_batch_data(
        start_time=date - pd.Timedelta(days=7),
        end_time=date
    )

    print(X.columns)

    print(f"Number of days retrieved from FV: {len(X)}")

    if len(X) < 7:
        raise RuntimeError(f"Expected 7 row from Feature View, got {len(X)}")

    return X

In [38]:
# Step 3
def run_models(X: pd.DataFrame, date) -> pd.DataFrame:
    mr = project.get_model_registry()
    
    results = []

    for h in range(1, MAX_HORIZON + 1):
        print(f"Running {MODEL_NAME}_h{h}")
        model_name = f"{MODEL_NAME}_h{h}"

        model = mr.get_model(
            name=model_name,
            version = 1
        )
        
        fv = model.get_feature_view()

        model_dir = model.download()

        clf = joblib.load(f"{model_dir}/model.pkl")

        y_proba = clf.predict_proba(X)[:, 1]

        result = {
            "timestamp": date + pd.Timedelta(days=h),
            "horizon_days": h,
            "probability": float(y_proba[0]),
        }

        results.append(result)

    # out_dir = Path(DATA_PATH)
    # out_dir.mkdir(parents=True, exist_ok=True)

    # pd.DataFrame(results).to_json(
    #     out_dir / "predictions.json",
    #     orient="records",
    #     indent=2
    # )

    out_dir = Path(root_dir) / "docs" / "aurora" / "assets"
    out_dir.mkdir(parents=True, exist_ok=True)

    pd.DataFrame(results).to_json(
        out_dir / "predictions.json",
        orient="records",
        indent=2
    )


    print(f"Saved {len(results)} predictions to {out_dir / 'predictions.json'}")


    return pd.DataFrame(results)


In [39]:
# Step 4
def save_predictions(df: pd.DataFrame):
    fg = fs.get_or_create_feature_group(
        name="aurora_predictions",
        version=1,
        primary_key=["timestamp", "horizon_days"],
        description="Daily batch aurora predictions",
    )

    fg.insert(df)

## Main Loop

In [40]:
def run_daily_inference():
    date = fetch_latest_raw_data()

    print("Date used one the fetch data raw", date)
    X = obtain_data_fv(date)

    X_engi = util.geomagnetic_feature_engineering(X)
    
    X_engi = X_engi.sort_values("date").tail(1).reset_index(drop=True)
    
    date = X_engi.loc[0, "date"]

    X_engi.drop(columns=['date'], inplace=True)
    
    predictions = run_models(X_engi, date)

    print(predictions)
    #save_predictions(predictions)


run_daily_inference()


2026-01-05 23:18:50,933 WARNING: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead

2026-01-05 23:18:51,723 INFO: 	17 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1289364/fs/1278019/fg/1893812


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: geomagnetic_daily_final_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1289364/jobs/named/geomagnetic_daily_final_1_offline_fg_materialization/executions
2026-01-05 23:19:12,676 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2026-01-05 23:19:15,925 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2026-01-05 23:20:59,928 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2026-01-05 23:21:00,114 INFO: Waiting for log aggregation to finish.
2026-01-05 23:21:15,776 INFO: Execution finished successfully.
Last data from kp: 2026-01-05 00:00:00
2026-01-05 23:21:16,469 INFO: 	3 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1289364/fs/1278019/fg/1893813


Uploading Dataframe: 100.00% |██████████| Rows 8/8 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: sweden_weather_daily_final_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1289364/jobs/named/sweden_weather_daily_final_1_offline_fg_materialization/executions
2026-01-05 23:21:34,353 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2026-01-05 23:21:40,847 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2026-01-05 23:23:18,159 INFO: Waiting for execution to finish. Current state: SUCCEEDING. Final status: SUCCEEDED
2026-01-05 23:23:18,331 INFO: Waiting for log aggregation to finish.
2026-01-05 23:23:30,577 INFO: Execution finished successfully.
Saved 5 days of context to context.json
Date used one the fetch data raw 2026-01-05 00:00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.46s) 
Index(['date', 'kp1', 'kp2', 'kp3', 'kp4', 'kp5', 'kp6', 'kp7', 'kp8', 'ap1',
       'ap2', 'ap3', 'ap4

...

Downloading: 0.000%|          | 0/462656 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/31184 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/11691 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/35011 elapsed<00:00 remaining<?

Saved 5 predictions to /Users/appbites/Desktop/id2223-project/docs/aurora/assets/predictions.json
                  timestamp  horizon_days  probability
0 2026-01-05 00:00:00+00:00             1     0.450832
1 2026-01-06 00:00:00+00:00             2     0.511205
2 2026-01-07 00:00:00+00:00             3     0.497482
3 2026-01-08 00:00:00+00:00             4     0.482068
4 2026-01-09 00:00:00+00:00             5     0.540067
